# Web Scraping

In [22]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
import numpy as np
import csv

verbose = True


## Elo Ratings

&#9632; For Fifa Ranking by year <br>
&#9632; Has data on Matches [Data can be grabbed from results.csv] <br> 
<ol>
    &#9633; Home/Away/Neutral <br>
    &#9633; Wins/Losses/Draws <br>
    </ol>

In [67]:
URL = 'https://eloratings.net/'
headers = {"User-Agent":"Mozilla/5.0"}
requested = requests.get(URL) # get URL
verbose = True

if verbose: # print raw html content
    print('Raw HTML content \n', requested.content, '\n\n')

soup = BeautifulSoup(requested.content, 'lxml')

if verbose:
    print('Prettified Soup \n', soup.prettify(), '\n\n')

Raw HTML content 
 b'<!DOCTYPE html>\n<html lang="en"><head><title>World Football Elo Ratings</title>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>\n<meta name="description" content="Ratings for national football teams based on the Elo rating system.">\n<meta name="keywords" content="football, ratings, Elo, rankings, national, international, soccer, teams">\n<meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0, user-scalable=no">\n<link rel="stylesheet" href="scripts/slick.grid.css" type="text/css"/>\n<link rel="stylesheet" href="scripts/dygraph.css" type="text/css"/>\n<script type="text/javascript" src="scripts/dygraph.js"></script>\n<script type="text/javascript" src="scripts/jquery.js"></script>\n<script type="text/javascript" src="scripts/slick.core.js"></script>\n<script type="text/javascript" src="scripts/slick.grid.js"></script>\n<script type="text/javascript" src="scripts/cldr.js"></script>\n<script t


## Transfer Market

&#9632; Squad Size <br>
&#9632; Average Age <br> 
&#9632; Total Value <br>
&#9632; Points <br>
&#9632; Confederation <br>
<ol>
    &#9633; Valuable b/c could get confederation data to use <br>
    &#9633; Potential Issue Confederation stats only, CONACAF not as good as UEFA <br>
    </ol>

In [87]:
def get_Fifa_Rankings(URL, rankings, headers): # function for grabbing fifa rankings from one page
    requested = requests.get(URL, headers = headers)
    soup = BeautifulSoup(requested.content, 'lxml')
    dom = etree.HTML(str(soup)) #. beautiful soup stuff
    
    base_path = '//*[@id="yw1"]/table/tbody/tr['
    xpath = '1]/td[2]/a[2]' # xpath addition
    # xpath = '//*[@id="yw1"]/table/tbody/tr[1]/td[2]/a[2]' for team name
    # xpath = '//*[@id="yw1"]/table/tbody/tr[1]/td[i]' for confederation(i = 3) for ranking(i=4)
    
    teams = 25
    currentRow = ['Name', 'Fed', 'Rank'] # blank template
    for i in range(teams):
        xpath = base_path + str(i+1) + ']/td['
        teamPath = xpath + '2]/a[2]'
        currentRow[0] = dom.xpath(teamPath)[0].text
        for j in range(1,3):
            path = xpath + str(j+2) + ']'
            currentRow[j] = dom.xpath(path)[0].text
        rankings.append(currentRow)
        currentRow = ['Name', 'Fed', 'Rank']
        
    #print(dom.xpath(xpath)[0].text)
    return rankings

def get_Next_Date(currentDate, newDate, baseURL, dateURL, headers):
    # format currentDate = ['year', 'month', 'day']
    currentDate[2] = int(currentDate[2])- 1
    if (int(currentDate[2]) == 0): # next month
        currentDate[2] = 31
        currentDate[1] = int(currentDate[1])-1

    if (int(currentDate[1]) == 0): # next year
        currentDate[1] = 12
        currentDate[0] = int(currentDate[0]) - 1

    newDate = ''
    testDate = currentDate
    for x in range(len(testDate)):
        if x ==2:
            newDate += str(testDate[x])
        else:
            newDate += str(testDate[x])+'-'
    
    requested = requests.get(baseURL + newDate + dateURL, headers = headers)
    while str(requested) == '<Response [500]>':
        # next date
        currentDate[2] = int(currentDate[2])- 1
        if (int(currentDate[2]) == 0): # next month
            currentDate[2] = 31
            currentDate[1] = int(currentDate[1])-1

        if (int(currentDate[1]) == 0): # next year
            currentDate[1] = 12
            currentDate[0] = int(currentDate[0]) - 1
            
        newDate = ''
        testDate = currentDate
        for x in range(len(testDate)):
            if x ==2:
                newDate += str(testDate[x])
            else:
                newDate += str(testDate[x])+'-'
        print(newDate)
        requested = requests.get(baseURL + newDate + dateURL, headers = headers)
        print(requested)
        
    return newDate
    #soup = BeautifulSoup(requedted.content, 'lxml')
    #dom = etree.HTML(str(soup))
    

In [89]:
baseURL = 'https://www.transfermarkt.us/statistik/weltrangliste/statistik/stat/ajax/yw1/datum/' # URL of website w/ chart or data
headers = {"User-Agent":"Mozilla/5.0"}
rankings = []
pages = 3
years = 3
date = ['2022', '03', '31']
for j in range(years):
    for i in range(pages):
        for x in range(len(date)):
            newDate = ''
            if x ==2:
                newDate += str(date[x])
            else:
                newDate += str(date[x])+'-'
        URL = (baseURL + newDate + '/plus/0/galerie/0/page/' + str(i+1))
        rankings = get_Fifa_Rankings(URL, rankings, headers)
    
    print(date)
    date = get_Next_Date(date, newDate, baseURL, '/plus/0/galerie/0/page/0', headers)
    print(date)
    print(rankings)
    rankings = []
    

['2022', '03', '31']
2022-03-29
<Response [500]>
2022-03-28
<Response [500]>
2022-03-27
<Response [500]>
2022-03-26
<Response [500]>
2022-03-25
<Response [500]>
2022-03-24
<Response [500]>
2022-03-23
<Response [500]>
2022-03-22
<Response [500]>
2022-03-21
<Response [500]>
2022-03-20
<Response [500]>
2022-03-19


KeyboardInterrupt: 

In [31]:
''' make it into a csv file
Format as follows
id | Rank | country_full | country_abbrev | Total points | Previous points | Rank change | Confederation | Rank date

'''
fields = ['Country', 'conference', 'ranking/points']
rows = rankings
with open('fifa_rankings.csv', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(rows)


## Fifa Index

&#9632; Get Fifa video game team rankings <br>
<ol>
    &#9633; ATT <br>
    &#9633; MID <br>
    &#9633; DEF <br>
    &#9633; OVR <br>
    &#9633; Team Rating <br>
    </ol>
    
&#9632; How much of this is UNIQUE data? <br>
&#9632; Easier to calculate OVR? <br>

In [18]:
def get_Fifa_Index(URL, teamRank):
    webpage = requests.get(URL)
    soup = BeautifulSoup(webpage.content, 'lxml')
    dom = etree.HTML(str(soup))
    # /a for team name and /span for int
    # tr[i]/td[j], where (i-2) is team rank, where j is column {2; Team Name, 4; ATT, 5; MID, 6; DEF, 7; OVR}  

    moreTeams = True
    currentRow = ['Name', 'ATT', 'MID', 'DEF', 'Date']
    i=0
    count = 0
    while moreTeams:

        teamVal = i+3
        xpath = '/html/body/main/div/div/div[2]/div[2]/table/tbody/tr[' + str(teamVal) + ']/td[2]/a'
        try:
            #print('try:', i)
            currentRow[0]=(dom.xpath(xpath)[0].text)
            for j in range(4, 7):
                xpath = '/html/body/main/div/div/div[2]/div[2]/table/tbody/tr[' + str(teamVal) + ']/td[' + str(j) + ']/span'
                #print(xpath)
                currentRow[j-3]=(dom.xpath(xpath)[0].text)
            currentRow[4] = dom.xpath('/html/body/main/div/div/div[2]/nav[1]/ol/li[3]/a')[0].text
            teamRank.append(currentRow)
            currentRow = ['Name', 'ATT', 'MID', 'DEF', 'Date']
            count = 0
            i +=1
        except:
            #print('except: ', i)
            count += 1
            i += 1
            if count == 5:
                break
            pass
    return teamRank

In [24]:
base_url = 'https://www.fifaindex.com/teams/fifa'
# URL = 'https://www.fifaindex.com/teams/fifa22/?page=1&type=1'
teamRank = []
year = 22
page = 1

while year > 4:
    if year < 10:
        year = '0'+str(year)
    else:
        year = str(year)
    URL = base_url + year + '/?page=' + str(page) + '&type=1'
    teamRank = get_Fifa_Index(URL, teamRank)
    year = int(year)
    if page == 1:
        page += 1
    else:
        page = 1
        year -= 1
    
#for x in range(len(teamRank)):
#    print('%i, %s, %s, %s, %s, %s' % (x, teamRank[x][0], teamRank[x][1], teamRank[x][2], teamRank[x][3], teamRank[x][4]))
    
fields = ['Country', 'ATT', 'MID', 'DEF', 'Date']

rows = teamRank

with open('test1.csv', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(rows)


## Fifa Ranking

&#9632; Total Points <br>
&#9632; Ranking <br>

In [6]:
URL = 'https://www.fifa.com/fifa-world-ranking/men?dateId=id13603'
webpage = requests.get(URL, headers={'Accept-Encoding': 'identity'})
soup = BeautifulSoup(webpage.text, 'lxml')
dom = etree.HTML(str(soup))
xpath = '//*[@id="content"]/main/section[2]/div/div/div[1]/table/tbody/tr[1]/td[3]/span[1]'
print(dom.xpath(xpath))


ContentDecodingError: ('Received response with content-encoding: gzip, but failed to decode it.', error('Error -3 while decompressing data: incorrect header check'))